In [104]:
import pandas as pd
from pathlib import Path

In [105]:
import numpy as np
from collections import Counter

In [106]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [107]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "Uploading_Data_Colab_1.xlsx" with length 9000 bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving clean_feature_set.csv to clean_feature_set (3).csv
Saving clean_wait_times.csv to clean_wait_times (4).csv
User uploaded file "Uploading_Data_Colab_1.xlsx" with length 9000 bytes
User uploaded file "Uploading_Data_Colab_1.xlsx" with length 9000 bytes


In [108]:
wait_times_path = Path('clean_wait_times.csv')
wait_times_df = pd.read_csv(wait_times_path)
wait_times_df.head()

,date,ride_name,sactmin
0,2015-01-01,7_dwarfs_train,54.5
1,2015-01-01,big_thunder_mtn,37.0
2,2015-01-01,haunted_mansion,27.5
3,2015-01-01,it_s_a_small_world,15.0
4,2015-01-01,splash_mountain,11.5


In [109]:
wait_times_df = wait_times_df.round({'sactmin':0})

In [110]:
wait_times_df

,date,ride_name,sactmin
0,2015-01-01,7_dwarfs_train,54.0
1,2015-01-01,big_thunder_mtn,37.0
2,2015-01-01,haunted_mansion,28.0
3,2015-01-01,it_s_a_small_world,15.0
4,2015-01-01,splash_mountain,12.0
...,...,...,...
16106,2021-12-28,haunted_mansion,12.0
16107,2021-12-28,splash_mountain,22.0
16108,2021-12-28,pirates_of_caribbean,21.0
16109,2021-12-28,7_dwarfs_train,30.0


In [111]:
feature_set_path = Path('clean_feature_set.csv')
feature_set_df = pd.read_csv(feature_set_path)
feature_set_df.head()

,date,day_week,day_year,week_year,month_year,year,prox_to_holiday,wdw_mean_temp,wdw_precip,schools_in_session,florida_schools_in_session
0,2015-01-01,5,0,0,1,2015,0,66.41,0.12,0.00,0.00
1,2015-01-02,6,1,0,1,2015,2,69.36,0.12,0.00,0.00
2,2015-01-03,7,2,0,1,2015,3,75.22,0.07,0.00,0.00
3,2015-01-04,1,3,1,1,2015,4,75.95,0.12,0.67,0.79
4,2015-01-05,2,4,1,1,2015,5,64.60,0.12,0.67,0.79


In [112]:
merged_data = wait_times_df.merge(feature_set_df, how='right', on='date')

In [113]:
merged_data.head()

,date,ride_name,sactmin,day_week,day_year,week_year,month_year,year,prox_to_holiday,wdw_mean_temp,wdw_precip,schools_in_session,florida_schools_in_session
0,2015-01-01,7_dwarfs_train,54.0,5,0,0,1,2015,0,66.41,0.12,0.0,0.0
1,2015-01-01,big_thunder_mtn,37.0,5,0,0,1,2015,0,66.41,0.12,0.0,0.0
2,2015-01-01,haunted_mansion,28.0,5,0,0,1,2015,0,66.41,0.12,0.0,0.0
3,2015-01-01,it_s_a_small_world,15.0,5,0,0,1,2015,0,66.41,0.12,0.0,0.0
4,2015-01-01,splash_mountain,12.0,5,0,0,1,2015,0,66.41,0.12,0.0,0.0


In [114]:
merged_data.dtypes

date                           object
ride_name                      object
sactmin                       float64
day_week                        int64
day_year                        int64
week_year                       int64
month_year                      int64
year                            int64
prox_to_holiday                 int64
wdw_mean_temp                 float64
wdw_precip                    float64
schools_in_session            float64
florida_schools_in_session    float64
dtype: object

In [115]:
merged_data['date'] = pd.to_datetime(merged_data['date'])

In [116]:
merged_data.dtypes

date                          datetime64[ns]
ride_name                             object
sactmin                              float64
day_week                               int64
day_year                               int64
week_year                              int64
month_year                             int64
year                                   int64
prox_to_holiday                        int64
wdw_mean_temp                        float64
wdw_precip                           float64
schools_in_session                   float64
florida_schools_in_session           float64
dtype: object

In [117]:
merged_data.describe()

,sactmin,day_week,day_year,week_year,month_year,year,prox_to_holiday,wdw_mean_temp,wdw_precip,schools_in_session,florida_schools_in_session
count,13210.000000,13217.000000,13217.000000,13217.000000,13217.000000,13217.000000,13217.000000,13217.000000,13217.000000,13217.000000,13217.000000
mean,22.544815,4.022547,183.341379,26.291065,6.564198,2017.214118,7.557842,73.665223,0.141189,0.682258,0.694854
std,15.810411,2.010890,109.211865,15.607384,3.571326,1.578649,6.515731,9.017332,0.076805,0.337521,0.409953
min,0.000000,1.000000,0.000000,0.000000,1.000000,2015.000000,0.000000,39.750000,0.030000,0.000000,0.000000
25%,12.000000,2.000000,86.000000,12.000000,3.000000,2016.000000,3.000000,68.320000,0.080000,0.440000,0.280000
50%,19.000000,4.000000,182.000000,26.000000,7.000000,2017.000000,6.000000,75.490000,0.120000,0.870000,0.970000
75%,28.000000,6.000000,282.000000,40.000000,10.000000,2019.000000,10.000000,81.070000,0.200000,0.940000,1.000000
max,176.000000,7.000000,365.000000,53.000000,12.000000,2020.000000,33.000000,87.760000,0.350000,0.970000,1.000000


In [118]:
merged_data_dropna = merged_data.dropna(how='any')

In [119]:
merged_data.corr(method='pearson')

,sactmin,day_week,day_year,week_year,month_year,year,prox_to_holiday,wdw_mean_temp,wdw_precip,schools_in_session,florida_schools_in_session
sactmin,1.000000,0.007499,-0.068270,-0.069655,-0.069640,0.138698,-0.056866,-0.074725,-0.029152,-0.068540,-0.070900
day_week,0.007499,1.000000,-0.010074,-0.028458,-0.010580,-0.001163,0.013281,-0.000486,0.009855,0.036729,0.026041
day_year,-0.068270,-0.010074,1.000000,0.999654,0.996733,0.021640,0.119195,0.256026,0.063588,-0.113431,-0.075776
week_year,-0.069655,-0.028458,0.999654,1.000000,0.996384,0.012107,0.119089,0.256214,0.063568,-0.114030,-0.075953
month_year,-0.069640,-0.010580,0.996733,0.996384,1.000000,0.022323,0.119184,0.259773,0.066193,-0.106497,-0.070504
year,0.138698,-0.001163,0.021640,0.012107,0.022323,1.000000,-0.052749,-0.101659,-0.056330,-0.006518,-0.008062
prox_to_holiday,-0.056866,0.013281,0.119195,0.119089,0.119184,-0.052749,1.000000,0.293649,0.341390,-0.119896,-0.136071
wdw_mean_temp,-0.074725,-0.000486,0.256026,0.256214,0.259773,-0.101659,0.293649,1.000000,0.597997,-0.293198,-0.264422
wdw_precip,-0.029152,0.009855,0.063588,0.063568,0.066193,-0.056330,0.341390,0.597997,1.000000,-0.497436,-0.491789
schools_in_session,-0.068540,0.036729,-0.113431,-0.114030,-0.106497,-0.006518,-0.119896,-0.293198,-0.497436,1.000000,0.948667


In [120]:
export_path = Path("../merged_data.json")

In [121]:
merged_data.to_json(export_path, orient='records', date_format='iso')

In [122]:
merged_data.dtypes

date                          datetime64[ns]
ride_name                             object
sactmin                              float64
day_week                               int64
day_year                               int64
week_year                              int64
month_year                             int64
year                                   int64
prox_to_holiday                        int64
wdw_mean_temp                        float64
wdw_precip                           float64
schools_in_session                   float64
florida_schools_in_session           float64
dtype: object

In [123]:
target = ["sactmin"]

Split Data into Training and Testing

In [124]:
#create features
X = (merged_data_dropna.drop(columns=['sactmin','date','ride_name']))


#create target
y = merged_data_dropna.loc[:,target].copy()

In [125]:
X.describe

<bound method NDFrame.describe of        day_week  day_year  week_year  month_year  year  prox_to_holiday  \
0             5         0          0           1  2015                0   
1             5         0          0           1  2015                0   
2             5         0          0           1  2015                0   
3             5         0          0           1  2015                0   
4             5         0          0           1  2015                0   
...         ...       ...        ...         ...   ...              ...   
13212         5       365         52          12  2020                0   
13213         5       365         52          12  2020                0   
13214         5       365         52          12  2020                0   
13215         5       365         52          12  2020                0   
13216         5       365         52          12  2020                0   

       wdw_mean_temp  wdw_precip  schools_in_session  \
0        

In [126]:
X.tail()

,day_week,day_year,week_year,month_year,year,prox_to_holiday,wdw_mean_temp,wdw_precip,schools_in_session,florida_schools_in_session
13212,5,365,52,12,2020,0,60.38,0.08,0.0,0.0
13213,5,365,52,12,2020,0,60.38,0.08,0.0,0.0
13214,5,365,52,12,2020,0,60.38,0.08,0.0,0.0
13215,5,365,52,12,2020,0,60.38,0.08,0.0,0.0
13216,5,365,52,12,2020,0,60.38,0.08,0.0,0.0


In [127]:
#check the balance of target values
y['sactmin'].value_counts()

12.0     578
14.0     568
16.0     566
18.0     533
10.0     509
        ... 
95.0       1
105.0      1
100.0      1
112.0      1
136.0      1
Name: sactmin, Length: 135, dtype: int64

In [128]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X_train.shape

(9907, 10)

In [129]:
from imblearn.ensemble import BalancedRandomForestClassifier
rf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
rf_model.fit(X_train, y_train)
print(Counter(y_train['sactmin']))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


Counter({14.0: 431, 12.0: 423, 16.0: 411, 18.0: 405, 10.0: 395, 20.0: 395, 15.0: 350, 13.0: 341, 22.0: 322, 17.0: 308, 21.0: 297, 11.0: 292, 8.0: 289, 24.0: 277, 19.0: 256, 26.0: 238, 9.0: 237, 23.0: 233, 28.0: 230, 6.0: 225, 30.0: 204, 25.0: 199, 27.0: 195, 7.0: 173, 29.0: 159, 32.0: 156, 4.0: 148, 5.0: 140, 36.0: 140, 34.0: 131, 31.0: 121, 3.0: 104, 33.0: 103, 38.0: 97, 35.0: 96, 2.0: 91, 37.0: 86, 40.0: 77, 42.0: 69, 44.0: 61, 48.0: 58, 43.0: 54, 41.0: 50, 39.0: 49, 46.0: 45, 50.0: 43, 1.0: 42, 45.0: 39, 54.0: 39, 49.0: 36, 52.0: 32, 47.0: 31, 51.0: 29, 55.0: 26, 56.0: 26, 60.0: 24, 57.0: 21, 58.0: 20, 64.0: 19, 53.0: 18, 0.0: 18, 66.0: 14, 61.0: 14, 62.0: 13, 70.0: 13, 74.0: 12, 68.0: 11, 71.0: 10, 59.0: 10, 67.0: 10, 76.0: 9, 69.0: 9, 63.0: 9, 65.0: 9, 80.0: 9, 78.0: 9, 77.0: 8, 83.0: 6, 84.0: 5, 72.0: 5, 85.0: 5, 101.0: 5, 79.0: 5, 81.0: 4, 75.0: 4, 88.0: 4, 106.0: 4, 73.0: 4, 92.0: 3, 89.0: 3, 94.0: 3, 90.0: 3, 128.0: 3, 91.0: 3, 98.0: 3, 116.0: 3, 86.0: 2, 111.0: 2, 93.0: 2, 82

In [130]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = rf_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


0.0017699034467069793

In [131]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [132]:
# Print the imbalanced classification report
# Balanced Random Forest Classifier
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.00      0.00      1.00      0.00      0.00      0.00         9
        1.0       0.00      0.00      0.99      0.00      0.00      0.00        14
        2.0       0.06      0.02      1.00      0.04      0.16      0.02        41
        3.0       0.00      0.00      1.00      0.00      0.00      0.00        36
        4.0       0.02      0.02      0.99      0.02      0.14      0.02        47
        5.0       0.00      0.00      0.99      0.00      0.00      0.00        36
        6.0       0.12      0.03      1.00      0.04      0.17      0.02        73
        7.0       0.05      0.04      0.99      0.04      0.19      0.03        52
        8.0       0.14      0.01      1.00      0.02      0.10      0.01       108
        9.0       0.11      0.01      1.00      0.02      0.11      0.01        83
       10.0       0.00      0.00      1.00      0.00      0.00      0.00       114
   

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Sensitivity is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [133]:
# List the features sorted in descending order by feature importance
importances = sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)
for importance in importances:
    print(f'{importance[1]}:  {importance[0]*100:.1f}%')

wdw_mean_temp:  14.5%
day_year:  13.4%
schools_in_session:  11.9%
prox_to_holiday:  11.0%
week_year:  10.8%
wdw_precip:  9.2%
day_week:  8.4%
year:  8.4%
florida_schools_in_session:  6.7%
month_year:  5.7%
